[toc]

# Tensorflow index_to_string_table_from_file和index_table_from_file

这两个函数的作用有点类似于 idx2label 和 label2idx

从文件中构建 `int->string` 映射和 `string->int` 映射，分别返回

`tf.contrib.lookup.HashTable` 对象和 `tf.contrib.lookup.IdTableWithHashBuckets`对象

但是 `index_to_string_table_from_file` 和 `index_table_from_file` 都返回的字符串是 byte 类型的

## index_to_string_table_from_file

In [16]:
!echo -e "B-DET\nB-TYPE\nE-DET\nE-TYPE\nI-DET\nI-TYPE\nO\nS-DET\nS-TYPE" > vocab.tags.txt

In [34]:
import tensorflow as tf 
indices = tf.constant([1, 5, 8, 11], tf.int64)
reverse_vocab_tags = tf.contrib.lookup.index_to_string_table_from_file('vocab.tags.txt') 
selected = reverse_vocab_tags.lookup(indices)
tables = reverse_vocab_tags.export()
with tf.Session() as sess:   
    tf.tables_initializer().run() 
    idx, byte_strings = sess.run(tables)  
    print(sess.run(selected))
    print(idx)
    print(byte_strings)

[b'B-TYPE' b'I-TYPE' b'S-TYPE' b'UNK']
[8 7 6 5 4 3 2 1 0]
[b'S-TYPE' b'S-DET' b'O' b'I-TYPE' b'I-DET' b'E-TYPE' b'E-DET' b'B-TYPE'
 b'B-DET']


注意，如果 index 超过了 table 中的单词个数，那么会返回 `UNK`

可以使用 `.decode` 方法转换为 unicode 字符

In [35]:
strings = [byte_string.decode() for byte_string in byte_strings]

### index_table_from_file

In [32]:
!echo -e "B-DET\nB-TYPE\nE-DET\nE-TYPE\nI-DET\nI-TYPE\nO\nS-DET\nS-TYPE" > vocab.tags.txt

注意，没有在 table 中的会被解码为 9

In [33]:
import tensorflow as tf

features = tf.constant(['B-DET', 'S-TYPE', 'O', 'haha'])  # print(type('工')) # str
reverse_vocab_tags = tf.contrib.lookup.index_table_from_file(
    'vocab.tags.txt', key_dtype=tf.dtypes.string, num_oov_buckets=1)

values = reverse_vocab_tags.lookup(features)
with tf.Session() as sess:
    tf.tables_initializer().run()
    print(values.eval())

[0 8 6 9]


# References

[Tensorflow：tf.contrib.lookup索引构建和查询_皮皮blog-CSDN博客](https://blog.csdn.net/pipisorry/article/details/90409062)